This notebook reads the network data created using "GC_based_mapping_of_Network.ipynb" notebook and basically analyse the communication paths between pre-defined nodes of interest.

In [ ]:
# Load the python package
import os
# import dynetan
from dynetan.toolkit import *
from dynetan.viz import *
from dynetan.proctraj import *
from dynetan.gencor import *
from dynetan.contact import *
from dynetan.datastorage import *

#from numpy.linalg import norm
from itertools import islice
# from scipy import stats

import pandas as pd
import numpy as np
import scipy as sp

import copy
import multiprocess as mp

In [ ]:
dnad = DNAdata()

# Load network data obtained by Dynetan module 

In [ ]:
## Path to load data from
dataDir = "/path/to/data/directory"

# Path where results will be written (you may want plots and data files in a new location)
workDir = "/path/to/out/directory"

fileNameRoot = "tgRNA_R973A_R1" ## from first notebook output
fullPathRoot = os.path.join(dataDir, fileNameRoot)

# Define the segID of the Ligand being studied.
ligandSegID = "SYST"
dnad.loadFromFile(fullPathRoot)
dcdVizFile = fullPathRoot + "_reducedTraj.dcd"
pdbVizFile = fullPathRoot + "_reducedTraj.pdb"

workUviz = mda.Universe(pdbVizFile, dcdVizFile)
# We add this to the object for ease of access.
dnad.nodesAtmSel = workUviz.atoms[ dnad.nodesIxArray ]
print(dnad.nodesAtmSel)

## Calculation of Signals and Noise:

A function to calculate pathlengths in terms of edge betweenness:

In [ ]:
def getBCsum(List, Indx):
        bc = 0
        # Iterate over edges in the path
        for i in range(len(List)-1):
            node1 = List[i]
            node2 = List[i+1]
            if node1 > node2:
                btw = (dnad.btws[Indx][( node2, node1)])/maximumBetweeness
            else:
                btw = (dnad.btws[Indx][( node1, node2)])/maximumBetweeness
            bc += btw
        return bc

Estimate of Noise (define source sink as resid-1):

In [ ]:
source = list(range(1153, 1211))  # List of residues to be considered source for Noise calculation (say the crRNA bases)
sink = list(range(0, 1153))  # List of residues to be considered sink (say all the Cas13 residues)


# Define how many extra sub-optimal paths will be written.
numSuboptimalPaths = 5

# Initialize variables.
minimumBetweeness = 100
maximumBetweeness = -1

for btw in dnad.btws[0].values():
    minimumBetweeness = min(minimumBetweeness, btw)
    maximumBetweeness = max(maximumBetweeness, btw)

# Normalize the value.
minimumBetweeness /= maximumBetweeness

# Open three files to update the sum of betweenness valus of paths with specific lengths (e.g. 6-8, 9-11, 12-14) --##
pathListFiles = [
    open(
        os.path.join(
            workDir, f"BCSum_pathlength{i}-{i+2}_fullcrRNANoise_tgRNA.dat"
        ),
        "w",
    )
    for i in range(6, 15, 3)
]


def BCSum_noise(args):
    srcNode, trgNode, winIndx = args

    tmpList = getSelFromNode(srcNode, dnad.nodesAtmSel, atom=True).split()
    srcNodeSel = "".join([tmpList[1], tmpList[4], tmpList[10]])

    tmpList = getSelFromNode(trgNode, dnad.nodesAtmSel, atom=True).split()
    trgNodeSel = "".join([tmpList[1], tmpList[4], tmpList[10]])

    # Reconstructs the optimal path from Floyd-Warshall algorithm
    pathFW = nx.reconstruct_path(srcNode, trgNode, dnad.preds[winIndx])

    for i, pathFile in enumerate(pathListFiles):
        if 6 + (3 * i) <= len(pathFW) <= 8 + (3 * i):
            BC = getBCsum(pathFW, winIndx)
            pathFile.write(str(BC) + "\n")

    # Behind the scenes, use Dijkstra algorithm to find sub-optimal paths
    for pathSO in islice(nx.shortest_simple_paths(dnad.nxGraphs[0], srcNode, trgNode, weight="dist"), 1, numSuboptimalPaths + 1):
        for i, pathFile in enumerate(pathListFiles):
            if 6 + (3 * i) <= len(pathSO) <= 8 + (3 * i):
                BC = getBCsum(pathSO, winIndx)
                pathFile.write(str(BC) + "\n")


# Run the function in parallel
with mp.Pool(processes=4) as pool:
    args_list = [
        (srcNode, trgNode, winIndx)
        for srcNode in source
        for trgNode in sink
        for winIndx in range(dnad.numWinds)
    ]
    pool.map(BCSum_noise, args_list)

for pathFile in pathListFiles:
    pathFile.close()



Estimate of Signals and write down the paths associated with the signals (define source sink as resid-1. When printing paths, original resid will be printed):

In [ ]:
def BCsum_signal(srcNode, trgNode, winIndx, dnad):
    tmpList = getSelFromNode(srcNode, dnad.nodesAtmSel, atom=True).split()
    srcNodeSel = "".join([tmpList[1], tmpList[4], tmpList[10]])

    tmpList = getSelFromNode(trgNode, dnad.nodesAtmSel, atom=True).split()
    trgNodeSel = "".join([tmpList[1], tmpList[4], tmpList[10]])

    normCorMat = copy.deepcopy(dnad.corrMatAll[winIndx, :, :])
    normCorMat /= normCorMat.max()

    allPaths = []

    pathFW = nx.reconstruct_path(srcNode, trgNode, dnad.preds[winIndx])
    allPaths.append(pathFW)

    for i, pathFile in enumerate(pathListFiles):
        if 6 + (3 * i) <= len(pathFW) <= 8 + (3 * i):
            BC = getBCsum(pathFW, winIndx)
            pathFile.write(str(BC) + "\n")
            

    for pathSO in islice(nx.shortest_simple_paths(dnad.nxGraphs[0], srcNode, trgNode, weight="dist"), 1, numSuboptimalPaths + 1):
        allPaths.append(pathSO)

        for i, pathFile in enumerate(pathListFiles):
            if 6 + (3 * i) <= len(pathSO) <= 8 + (3 * i):
                BC = getBCsum(pathSO, winIndx)
                pathFile.write(str(BC) + "\n")
                

    writePathDetails(allPaths, winIndx, dnad.btws[winIndx], normCorMat, pathway)



def writePathDetails(allPaths, winIndx, btws, normCorMat, pathway):
    for pathIndx, pathIter in enumerate(allPaths):
        for i in range(len(pathIter) - 1):
            node1 = pathIter[i] + 1
            node2 = pathIter[i + 1] + 1

            try:
                if node1 > node2:
                    btw = btws[(node2, node1)]
                else:
                    btw = btws[(node1, node2)]
            except KeyError:
                btw = minimumBetweeness

            string = "{} {} {} {} {}".format(
                node1,
                node2,
                normCorMat[node1, node2],
                btw / maximumBetweeness,
                pathIndx)
            pathway.write(string + "\n")



In [ ]:
source = list(range(1179,1185)) # List of residues to be considered source for signal calculation (say the "seed" bases)
sink = [1047, 1052, 471, 476] # List of residues to be considered sink for signal calculation (say the Catalytic residues)

# Initialize variables.
minimumBetweeness = 100
maximumBetweeness = -1

for btw in dnad.btws[0].values():
    minimumBetweeness = min(minimumBetweeness, btw)
    maximumBetweeness = max(maximumBetweeness, btw)

# Normalize the value.
minimumBetweeness /= maximumBetweeness

# Define how many extra sub-optimal paths will be written.
numSuboptimalPaths = 5

# open three files to update the sum of betweenness valus of signalling paths with specific lengths (e.g. 6-8, 9-11, 12-14) --##
pathListFiles = [
    open(
        os.path.join(
            workDir, f"BCSum_pathlength{i}-{i+2}_seedTocat_tgRNA.dat"
        ),
        "w",
    )
    for i in range(6, 15, 3)
]

## Open a file to write all the paths calculated between pair of nodes or group of nodes defined as source and sink
pathway = open(
        os.path.join(
            workDir, "seedTocat_allpaths_tgRNA.dat"
        ),
        "w",
    )

# Run the signal calculation for each pair of source and sink nodes
for srcNode in source:
    print(srcNode)
    for trgNode in sink:
        for winIndx in range(dnad.numWinds):
            BCsum_signal(srcNode, trgNode, winIndx, dnad)
            
for File in pathListFiles:
    File.close()
pathway.close()

## Plotting and Calculating SNR

Load Noise and Signal data:

In [ ]:
## Path to load data from
dataDir = "/path/to/data/directory"

# Load Noise data
noise_data = {}
for i in range(6, 15, 3):
    filename = f"BCSum_pathlength{i}-{i+2}_fullcrRNANoise_tgRNA.dat" ## change it accordingly filename pattern
    noise_data[f"sys_{i}_{i+2}"] = np.loadtxt(os.path.join(dataDir, filename))

# Load Signal data
signal_data = {}
path_lengths = [(6, 8), (9, 11), (12, 14)]
path_types = ["swtoc", "setoc"] ## change the labels based on file name
for path_length in path_lengths:
    for path_type in path_types:
        filename = f"BCSum_pathlength{path_length[0]}-{path_length[1]}_{path_type}_tgRNA.dat"
        signal_data[f"sys_{path_length[0]}_{path_length[1]}_{path_type}"] = np.loadtxt(os.path.join(dataDir, filename))

# Noise:
sys_6_8 = noise_data["sys_6_8"]
sys_9_11 = noise_data["sys_9_11"]
sys_12_14 = noise_data["sys_12_14"]

# Signal:
sys_6_8_swtoc = signal_data["sys_6_8_swtoc"]
sys_9_11_swtoc = signal_data["sys_9_11_swtoc"]
sys_12_14_swtoc = signal_data["sys_12_14_swtoc"]

sys_6_8_setoc = signal_data["sys_6_8_setoc"]
sys_9_11_setoc = signal_data["sys_9_11_setoc"]
sys_12_14_setoc = signal_data["sys_12_14_setoc"]



Plot:

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sb
from matplotlib import rcParams

# Path to Plot directory
PlotDir = "/path/to/plot/directory"

# Plot settings
%matplotlib inline
%config InlineBackend.figure_format = 'retina'  # high resolution
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Arial']

fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(7, 14), sharex=True, gridspec_kw={'hspace': 0.3})
(ax1, ax2, ax3) = axes

# Plot KDEs
sb.kdeplot(sys_6_8, color="lightgrey", common_norm=False, shade=True, alpha=0.8, ax=ax1, linewidth=0.5, edgecolor='black')
sb.kdeplot(sys_9_11, color="lightgrey", common_norm=False, shade=True, alpha=0.8, ax=ax2, linewidth=0.5, edgecolor='black')
sb.kdeplot(sys_12_14, color="lightgrey", common_norm=False, shade=True, alpha=0.8, ax=ax3, linewidth=0.5, edgecolor='black')

sb.kdeplot(sys_6_8_swtoc, color="#3C86F7", shade=True, alpha=0.5, ax=ax1, linewidth=0.5, edgecolor='black', common_norm=False)
sb.kdeplot(sys_6_8_setoc, color="#6BE0A6", shade=True, alpha=0.5, ax=ax1, linewidth=0.5, edgecolor='black', common_norm=False)

# axes properties
for ax in [ax1, ax2, ax3]:
    ax.set_ylim([0, 1.5])
    ax.set_xlim([-1, 5])
    ax.set_ylabel('Density', fontsize=24, fontweight='bold', labelpad=6)
    ax.set_xticks(list(range(-1, 6)))
    ax.tick_params(axis='both', labelsize=28, length=9, width=1.5)
    for axis in ['top', 'bottom', 'left', 'right']:
        ax.spines[axis].set_linewidth(1.3)  

ax1.set_title("Shorter paths", fontsize=20, fontweight='bold', pad=11)
ax2.set_title("Medium-length paths", fontsize=20, fontweight='bold', pad=11)
ax3.set_title("Longer paths", fontsize=20, fontweight='bold', pad=11)

# Save the plot
plt.tight_layout()
plt.savefig(os.path.join(PlotDir, 'outfilename.png'), dpi=600, bbox_inches="tight")


# Calculate SNR

In [ ]:
from scipy.integrate import quad
from sklearn.neighbors import KernelDensity

def mean_var_ratio(KDE):
    pdf = lambda x: np.exp(KDE.score_samples([[x]]))[0]
    mean_integration = quad(lambda x: x * pdf(x), a=-np.inf, b=np.inf)[0]
    variance_integration = quad(lambda x: (x ** 2) * pdf(x), a=-np.inf, b=np.inf)[0] - mean_integration ** 2
    return mean_integration / variance_integration

def calculate_snr(sys_data, kde_noise):
    try:
        if sys_data.shape[0] != 0:
            kde_sys = KernelDensity(kernel='gaussian', bandwidth=0.5).fit(sys_data[:, np.newaxis])
            mean_var_ratio_noise = mean_var_ratio(kde_noise)
            mean_var_ratio_signal = mean_var_ratio(kde_sys)
            snr_signal = mean_var_ratio_signal / mean_var_ratio_noise
            return mean_var_ratio_noise, mean_var_ratio_signal, snr_signal
        else:
            return None, None, None
    except IndexError:
        sys_data = np.array([sys_data])
        kde_sys = KernelDensity(kernel='gaussian', bandwidth=0.5).fit(sys_data[:, np.newaxis])
        mean_var_ratio_noise = mean_var_ratio(kde_noise)
        mean_var_ratio_signal = mean_var_ratio(kde_sys)
        snr_signal = mean_var_ratio_signal / mean_var_ratio_noise
        return mean_var_ratio_noise, mean_var_ratio_signal, snr_signal

kde_sys_6_8 = KernelDensity(kernel='gaussian', bandwidth=0.5).fit(sys_6_8[:, np.newaxis]) ## Noise data
# kde_sys_9_11 = KernelDensity(kernel='gaussian', bandwidth=0.5).fit(sys_9_11[:, np.newaxis]) ## Noise data
# kde_sys_12_14 = KernelDensity(kernel='gaussian', bandwidth=0.5).fit(sys_12_14[:, np.newaxis]) ## Noise data

sys_list = [sys_6_8_swtoc, sys_6_8_setoc]
for i, sys_data in enumerate(sys_list):
    try:
        print("Running...", i)
        mean_var_ratio_noise, mean_var_ratio_signal, snr_signal = calculate_snr(sys_data, kde_sys_6_8)
        if mean_var_ratio_noise is not None:
            print("mean_var_ratio of noise:", mean_var_ratio_noise)
            print("mean_var_ratio of signal:", mean_var_ratio_signal)
            print("SNR signal:", snr_signal)
        else:
            print("Got an empty dataset")
    except Exception as e:
        print("Error:", str(e))


## Calculate occurance of nodes across Optimal and Sub-optimal pathways considered as signals

In [ ]:
## Path to load data from
dataDir = "/path/to/data/directory"

s1 = np.loadtxt(os.path.join(dataDir,'filename_of_pathfile_as calculated_in_cell_7')) ## e.g. seedTocat_allpaths_tgRNA.dat
print(s1.shape)

Calculate occurence of residues in top-ranked paths and write the node frequencies:

In [ ]:
# List of source and sink residues corresponding to each signal considered (*original resid)
# E.g., for signals corresponding to the 'seed' region and 'catalytic site', provide all the residues corresponding to 'seed' and all 4 catalytic residues

tg_source_sink = [1048, 1053, 477, 472, 1192, 1193, 1194, 1195, 1196, 1197]  # seed-to-catalytic site

path_ranks = [0, 1, 2, 3, 4, 5]  # ranks of paths to be read from the path file
# path_ranks = list(range(51))  # ranks of paths

with open(os.path.join(dataDir, 'filename_to_write_nodefrequencies.dat'), 'w') as h:
    resid = [int(s1[i, 1]) for i in range(s1.shape[0]) if s1[i, 4] in path_ranks]
    print("shape of residue list:", len(resid))
    unique_resid = list(set((resid)))
    print("No of unique residues in the list:", len(unique_resid))
    print("Unique residues are:", unique_resid)
    for x in unique_resid:
        if x not in tg_source_sink:  # Excluding the source and sink resids
            print(x, resid.count(x), file=h) 


 Sort the occurence file in order of ascending residue ids: 

In [ ]:
! sort -n -k1 /path/to/data/directory/filename_to_write_nodefrequencies.dat > /path/to/data/directory/filename_to_write_nodefrequencies_sorted.dat
s2 = np.loadtxt(os.path.join(dataDir, 'filename_to_write_nodefrequencies_sorted.dat'))



In [ ]:
# List of residues in the calculated paths
resid = [str(s2[i, 0]) for i in range(s2.shape[0])]
print("\nList of residues in the calculated paths:", [int(float(i)) for i in resid])

# Label the bases according to nucleic acid naming for plotting purposes
crRNAlist = ['A(-8)', 'C(-7)', 'U(-6)', 'A(-5)', 'A(-4)', 'A(-3)', 'A(-2)', 'C(-1)', 'A1', 'C2', 'A3', 'A4', 'A5', 'U6', 'C7', 'U8', 'A9', 'U10', 'C11', 'U12', 'G13', 'A14', 'A15', 'U16', 'A17', 'A18', 'A19', 'C20', 'U21', 'C22', 'U23']  # tgRNA sys
crRNAresidlist = list(range(1176, 1207))

tgRNAlist = ['U(9*)', 'A(8*)', 'G(7*)', 'A(6*)', 'U(5*)', 'U(4*)', 'U(3*)']
tgRNAresidlist = list(range(1232, 1239))

label = []
for i in resid:
    i_int = int(float(i))
    if i_int < 1154:
        label.append(str(i_int))
    elif 1175 < i_int < 1207:
        label.append(crRNAlist[crRNAresidlist.index(i_int)])
    else:
        label.append(tgRNAlist[tgRNAresidlist.index(i_int)])

print("\nLabel of resids:", label)

# Occurrence of the residues
count = [s2[i, 1] for i in range(s2.shape[0])]
print("\nOccurrence of the residues:", count)

In [ ]:
# Path to Plot directory
PlotDir = "/path/to/plot/directory"

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'  # high resolution

# Plotting settings
barWidth = 0.7
plt.rcParams['axes.linewidth'] = 1.3

# Define colors based on Domain color
color1 = ['#F3B2DF' for i in resid if 360 < int(float(i)) < 509]  # HEPN1-I
color2 = ['#F1EFEF' for i in resid if 508 < int(float(i)) < 752]  # Helical-2
color3 = ['#F6C2CB' for i in resid if 751 < int(float(i)) < 814]  # HEPN1-II
color4 = ['#4691F3' for i in resid if 813 < int(float(i)) < 947]  # Linker
color5 = ['#68D4A7' for i in resid if 946 < int(float(i)) < 1154]  # HEPN2
color6 = ['#EB9B3F' for i in resid if 1153 <int(float(i)) < 1210] ## crRNA 
color7 = ['#D75CE2' for i in resid if 1211 <int(float(i)) < 1241] ## tgRNA
color_dom = color1 + color2 + color3 + color4 + color5 + color6 + color7

plt.figure(figsize=(14, 7))
plt.bar(resid, count, color=color_dom, width=barWidth, edgecolor='black', linewidth=1.0)
plt.ylim(0, 110)
plt.tick_params(axis='both', length=9, width=1.5)

plt.xlabel("Residues", fontname='Arial', fontsize=27, fontweight='bold')
plt.ylabel("Occurrence in Optimal Paths", fontname='Arial', fontsize=28, fontweight='bold')
plt.yticks(fontname='Arial', fontsize=28)

plt.xticks([r for r in range(len(resid))],
           [x for x in label], fontsize=24, rotation=90)

plt.legend(prop={"size": 28}, frameon=False)
plt.savefig(os.path.join(PlotDir,"filename_to_write_nodefrequencies_sorted.png", dpi=600, bbox_inches='tight')

## Calculate path-length across all optimal and sub-optimal pathways between pair of source-sink

In [ ]:
import math

def subpathlength(data, path_id_list, verbose=True):
    pathlengths = []
    edgecount = []
    d = 0
    c = 0
    
    for j in path_id_list:
        for i in range(data.shape[0]):
            if data[i, 4] == j:
                if data[i, 2] != 0:  # in terms of distance, i.e., -log(GC)
                    d += -math.log10(data[i, 2])  # in terms of distance, i.e., -log(GC)
                c += 1
            elif d != 0:
                pathlengths.append(d)
                edgecount.append(c)
                d = 0
                c = 0
            else:
                d = 0
                c = 0
    
    if verbose:
        print("Shape of pathlengths list:", len(pathlengths))
        print("Number of edges:", edgecount)
        print("Path lengths (weights of edges):", pathlengths)
    
    average_path_length = sum(pathlengths) / len(pathlengths) if len(pathlengths) > 0 else 0
    average_edge_count = sum(edgecount) / len(edgecount) if len(edgecount) > 0 else 0
    
    print("Average path length (betweenness estimate, path length estimate):", average_path_length, average_edge_count)
    
    return pathlengths, edgecount


In [ ]:
## Path to load data from
dataDir = "/path/to/data/directory"

s1 = np.loadtxt(os.path.join(dataDir,'filename_of_pathfile_as calculated_in_cell_7')) ## e.g. seedTocat_allpaths_tgRNA.dat
print(s1.shape)

subpathID = list(range(6)) ## Give ID of path ranks ; if want to get lengths of only the optimal path, subpathID = [0]


pathlength = subpathlength(s1,subpathID,verbose=False)
print(pathlength)

In [ ]:
import matplotlib.pyplot as plt
mport seaborn as sb
%matplotlib inline
%config InlineBackend.figure_format = 'retina'  # high resolution

fig = plt.figure(figsize=(8,6))

s3 = np.loadtxt('/Users/souviksinha/Desktop/Palermo_Lab/LabWork/Project_Cas13a/LbuCas13a-Antitag/run/Network_Analysis/atgRNA/seedToCatres_allpaths.dat')
s2 = np.loadtxt('/Users/souviksinha/Desktop/Palermo_Lab/LabWork/Project_Cas13a/LbuCas13a-crRNA-target/run/Network_Analysis/tgRNA/seedToCatres_allpaths.dat')
s1 = np.loadtxt('/Users/souviksinha/Desktop/Palermo_Lab/LabWork/Project_Cas13a/LbuCas13a-crRNA/run/Network_Analysis/crRNA/seedToCatres_allpaths.dat')
# subpathID = list(range(51)) ## Give ID of path ranks ; if want to get lengths of only the optimal path, subpathID = [0]
subpathID = list(range(6))

pathlength_cr = subpathlength(s1,subpathID,verbose=False)
pathlength_tg = subpathlength(s2,subpathID,verbose=False)
pathlength_atg = subpathlength(s3,subpathID,verbose=False)

ax = sb.kdeplot(pathlength_cr[1], color="springgreen", shade=True, alpha=0.5, linewidth=1, label = 'crRNA')
ax = sb.kdeplot(pathlength_tg[1], color="mediumpurple", shade=True, alpha=0.5, linewidth=1, label = 'tgRNA')
ax = sb.kdeplot(pathlength_atg[1], color="lightcoral", shade=True, alpha=0.5, linewidth=1, label = 'atgRNA')

# Border Width
for axis in ['top', 'bottom', 'left', 'right']:
    ax.spines[axis].set_linewidth(1.3)  # change width
ax.tick_params(axis='both', length=9, width=1.5)

plt.xlim(2, 20)
plt.ylim(0, 0.35)

plt.xlabel("Pathlengths", fontname='Arial', fontsize=28, fontweight='bold')
plt.ylabel("Density", fontname='Arial', fontsize=28, fontweight='bold')
plt.xticks([r for r in list(range(2, 21, 4))], [r for r in list(range(2, 21, 4))], fontname='Arial', fontsize=26)
plt.yticks([r for r in np.divide(np.arange(0, 36, 10), 100).tolist()],
           [r for r in np.divide(np.arange(0, 36, 10), 100).tolist()], fontname='Arial', fontsize=26)
plt.legend(prop={"size": 24}, bbox_to_anchor=(1.03, 1), frameon=False)

plt.tight_layout()
plt.savefig(os.path.join(PlotDir,"Edgecount_KDE.png", dpi=600)